#[PyTorch Tutorial](https://pytorch.org/tutorials/index.html)

In [1]:
%matplotlib inline

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

## 1. Load and normalizing the datasets using torchvision

The output of torchvision datasets are PILImage images of range [0, 1].
We transform them to Tensors of normalized range [-1, 1].



In [2]:
transform_train = transforms.Compose([
    #########################################
    #
    #
    #               your code
    #
    # Flip
    # Translation
    # Rotation
    # Crop
    # Resize
    # Lighting
    #
    # Please See, https://pytorch.org/docs/stable/torchvision/transforms.html
    #
    # CIFAR-10 input size : (3 x 32 x 32)
    # Desired Model input size : (3 x 32 x 32)
    # 
    # The size after transformation must be (3 x 32 x 32)
    #
    # In case Crop, giving some padding is useful. (can be implemented in one line)
    # In case Resize, cropping after resizing is needed. (can be implemented in one line)
    #
    #
    #########################################
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=512,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## 2. Define LeNet-5



In [3]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(576, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = torch.relu(x)
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)

        return x

In [4]:
net = LeNet5()
net = net.cuda()

## 3. Define a Loss function and optimizer

Let's use a Classification Cross-Entropy loss and SGD with momentum.



In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

## 4. Train the network

This is when things start to get interesting.
We simply have to loop over our data iterator, and feed the inputs to the
network and optimize.



In [6]:
import time

epochs = 25

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    start = time.time()

    correct = 0
    total = 0
    
    for i, data in enumerate(trainloader):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # print statistics
        running_loss += loss.item()

    end = time.time()
    print('(Train) [epoch : %d] loss: %.3f / time: %.3f / acc@1: %.3f' %
          (epoch + 1, running_loss / len(trainloader), (end-start), 100 * correct / total))
        
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('(Test) Accuracy of the network on the 10000 test images: %.3f %%\n' % (
        100 * correct / total))

print('Finished Training')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


(Train) [epoch : 1] loss: 2.271 / time: 12.891 / acc@1: 17.258
(Test) Accuracy of the network on the 10000 test images: 24.630 %

(Train) [epoch : 2] loss: 1.921 / time: 12.918 / acc@1: 30.652
(Test) Accuracy of the network on the 10000 test images: 37.480 %

(Train) [epoch : 3] loss: 1.642 / time: 13.033 / acc@1: 40.882
(Test) Accuracy of the network on the 10000 test images: 44.380 %

(Train) [epoch : 4] loss: 1.487 / time: 12.995 / acc@1: 46.432
(Test) Accuracy of the network on the 10000 test images: 48.790 %

(Train) [epoch : 5] loss: 1.375 / time: 12.916 / acc@1: 50.836
(Test) Accuracy of the network on the 10000 test images: 52.570 %

(Train) [epoch : 6] loss: 1.289 / time: 12.926 / acc@1: 54.070
(Test) Accuracy of the network on the 10000 test images: 54.620 %

(Train) [epoch : 7] loss: 1.231 / time: 12.901 / acc@1: 56.460
(Test) Accuracy of the network on the 10000 test images: 56.080 %

(Train) [epoch : 8] loss: 1.173 / time: 12.989 / acc@1: 58.674
(Test) Accuracy of the netw

KeyboardInterrupt: ignored